In [4]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

from tqdm import tqdm_notebook
import time

import csv

In [5]:
def write_csv(data:dict):
    """
    Функция записи полученных данных в файл
    """
    
    with open('data_reviews.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow((data['reviewText'], 
                             data['label']))
    
    
def get_parse(url:str, start_page:int = 0, end_page:int = 500):
    """
    Функция парсинга сайта 
    """
    
    # Использование selenium для решения капч, создание окна
    driver = webdriver.Firefox()
    # Циклом осуществляется проход по страницам с тизерами отзывов  
    for i in tqdm_notebook(range(start_page, end_page + 1)):
         
        if i % 10 == 1:
            time.sleep(5)
        try:
            # Получение данных с конкретной страницы с тизерами отзывов
            url = f'{str(url)}/{i + 1}/'
            driver.get(url)
            # Получение всех ссылок на полные отзывы на странице с тизерами
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            href_list = soup.find_all('a', class_='review-btn review-read-link')

            for link in href_list:
                # Проход циклом по всем ссылкам, полученным со страницы
                href = str(HOST + link['href'])
                driver.get(href)
                time.sleep(.5)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                # Получение текста отзыва 
                review_text = soup.find('div', 
                                        class_='review-body description').getText()
                # Получение метки отзыва
                label = soup.find('div', class_='rating-score tooltip-right')  \
                    .find('span').text
                data = {'reviewText': review_text, 'label': label}
                # Запись данных в файл
                write_csv(data)
                
        except Exception as ex:
            print(ex)

In [7]:
HOST = "https://otzovik.com"
URL = "https://otzovik.com/reviews/ozon_ru_online_shop/10/"

get_parse(URL, start_page=50, end_page=52)

C:\Users\Andrey\AppData\Local\Temp\ipykernel_10532\1391730791.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(start_page, end_page + 1)):


  0%|          | 0/3 [00:00<?, ?it/s]

In [38]:
df = pd.read_csv('data_reviews.csv', 
                 names=['reviewText', 'label'], 
                 encoding='windows_1251')
df

,reviewText,label
0,Впервые я взяла рассрочку(никогда ничего не бр...,1
1,"Только что увидела дешевый флэш накопитель, оп...",3
2,С данным интернет-магазином я познакомился год...,5
3,"Наша жизнь не стоит на месте. Время идет, и вс...",4
4,Интернет-магазин Ozon.ru .Этот интернет магази...,5
...,...,...
15399,"Везут товар хрен знает сколько, только завтрак...",1
15400,Заказывал подушку с доставкой на след. день в ...,1
15401,Неделю назад я совершила покупку на сайте мага...,1
15402,Уже в отчаянии адекватно и правильно решить пр...,1
